In [1]:
import torch.nn as nn
import torch
import math

batch_size = 1
image_size = 14
dim_in = 3
emb_dim = 16
c_group = 1
kernel_size = 3
stride = 2
c_group = 1

image = torch.zeros(batch_size, dim_in, image_size, image_size)
for b in range(batch_size):
    for c in range(dim_in):
        for i in range(image_size):
            for j in range(image_size):
                image[b, c, i, j] = (b+c+i+j) % 256

kernel = torch.zeros(emb_dim, dim_in, kernel_size, kernel_size)
kernel[:, :, :, :] = 0.1

c_group = math.gcd(dim_in, emb_dim) if c_group == 1 else c_group
padding = math.ceil(((kernel_size - 1) * 1 + 1 - stride) / 2)
print('padding: ', padding)
conv = nn.Conv2d(dim_in, emb_dim, kernel_size, stride, padding, dilation=1, groups=c_group, bias=False)
conv.weight.data = kernel
image = conv(image)
norm = nn.BatchNorm2d(emb_dim)
result = norm(image)

gamma = norm.weight
beta = norm.bias

#print(image.shape)
#print(image[0, 0, :, :])

print(gamma)
print(beta)
print(result[0, 0, :, :])

padding:  1
Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       requires_grad=True)
tensor([[-1.8920, -1.6551, -1.4365, -1.2178, -0.9992, -0.7805, -0.5619],
        [-1.6551, -1.2178, -0.8898, -0.5619, -0.2339,  0.0941,  0.4221],
        [-1.4365, -0.8898, -0.5619, -0.2339,  0.0941,  0.4221,  0.7500],
        [-1.2178, -0.5619, -0.2339,  0.0941,  0.4221,  0.7500,  1.0780],
        [-0.9992, -0.2339,  0.0941,  0.4221,  0.7500,  1.0780,  1.4060],
        [-0.7805,  0.0941,  0.4221,  0.7500,  1.0780,  1.4060,  1.7340],
        [-0.5619,  0.4221,  0.7500,  1.0780,  1.4060,  1.7340,  2.0619]],
       grad_fn=<SliceBackward0>)
